In [1]:
import os
import cv2
import numpy as np
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.feature import local_binary_pattern,hog
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [2]:
import os
import cv2
import numpy as np
from skimage.feature import  local_binary_pattern,graycomatrix, graycoprops

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# ===============================
# Parameter LBP
# ===============================
radius = 3
n_points = 8 * radius

# ===============================
# Fungsi ekstraksi fitur
# ===============================
def extract_features(image):
    # Pastikan gambar grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # --- LBP ---
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    lbp_hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, n_points+3), range=(0, n_points+2))
    lbp_hist = lbp_hist.astype('float')
    lbp_hist /= (lbp_hist.sum() + 1e-6)  # Normalisasi
    
    # --- GLCM ---
    glcm = graycomatrix(gray, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0,0]
    dissimilarity = graycoprops(glcm, 'dissimilarity')[0,0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0,0]
    energy = graycoprops(glcm, 'energy')[0,0]
    correlation = graycoprops(glcm, 'correlation')[0,0]
    
    glcm_features = np.array([contrast, dissimilarity, homogeneity, energy, correlation])

    # --- GLCM Multi Angle---
    #angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    #glcm = graycomatrix(gray, distances=[1], angles=angles, levels=256, symmetric=True, normed=True)

    #contrast     = np.mean(graycoprops(glcm, 'contrast')[0, :])
    #dissimilarity= np.mean(graycoprops(glcm, 'dissimilarity')[0, :])
    #homogeneity  = np.mean(graycoprops(glcm, 'homogeneity')[0, :])
    #energy       = np.mean(graycoprops(glcm, 'energy')[0, :])
    #correlation  = np.mean(graycoprops(glcm, 'correlation')[0, :])

    #glcm_features = np.array([contrast, dissimilarity, homogeneity, energy, correlation])

    

    # ======================
    # HOG
    # ======================
    #hog_feat = hog(
    #    gray,
    #    orientations=8,
    #    pixels_per_cell=(16, 16),
    #    cells_per_block=(2, 2),
    #    block_norm='L2-Hys',
    #    feature_vector=True
    #)
    
    # Gabungkan fitur LBP + GLCM + HOG
    #features = np.hstack([lbp_hist, glcm_features,hog_feat])
    #features = np.hstack([ glcm_features,hog_feat])
    features = np.hstack([ lbp_hist,glcm_features])
    return features

# ===============================
# Load dataset
# ===============================
def load_dataset(folder_tb, folder_normal):
    X, y = [], []
    for img_name in os.listdir(folder_tb):
        img_path = os.path.join(folder_tb, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            X.append(extract_features(img))
            y.append(1)  # label TB
    
    for img_name in os.listdir(folder_normal):
        img_path = os.path.join(folder_normal, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            X.append(extract_features(img))
            y.append(0)  # label normal
            
    return np.array(X), np.array(y)

# ===============================
# Tentukan folder               #
# ===============================
folder_tb = r'D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Train\tb'
folder_normal = r'D:\project\python\2025\tbcdetect\dataset2\TB_Chest_Radiography_Database\Train\normal'

X, y = load_dataset(folder_tb, folder_normal)

# ===============================
# Split dataset
# ===============================
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)




In [3]:

# ===============================
# Random Forest
# ===============================
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))

# ===============================
# SVM dengan normalisasi
# ===============================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC(kernel='rbf', C=1.0 , random_state=42)
svm.fit(X_train_scaled, y_train)
y_pred_svm = svm.predict(X_test_scaled)
print("SVM Accuracy:", accuracy_score(y_test, y_pred_svm))

# ===============================
# KNN
# ===============================
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
y_pred_knn = knn.predict(X_test_scaled)
print("KNN Accuracy:", accuracy_score(y_test, y_pred_knn))

Random Forest Accuracy: 0.9916666666666667
SVM Accuracy: 0.9892857142857143
KNN Accuracy: 0.9845238095238096


In [6]:
# ===============================
# 1. RANDOM FOREST
# ===============================
print("\n=== RANDOM FOREST ===")
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, rf_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, rf_pred))
print("\nClassification Report:\n", classification_report(y_test, rf_pred))

# ===============================
# 2. SVM (dengan normalisasi)
# ===============================
print("\n=== SVM (Normalized) ===")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

svm = SVC(kernel='rbf', C=1.0, gamma='scale')
svm.fit(X_train_scaled, y_train)
svm_pred = svm.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, svm_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, svm_pred))
print("\nClassification Report:\n", classification_report(y_test, svm_pred))

# ===============================
# 3. KNN (butuh normalisasi juga)
# ===============================
print("\n=== KNN ===")
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
knn_pred = knn.predict(X_test_scaled)

print("Accuracy:", accuracy_score(y_test, knn_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, knn_pred))
print("\nClassification Report:\n", classification_report(y_test, knn_pred))


=== RANDOM FOREST ===
Accuracy: 0.9916666666666667

Confusion Matrix:
 [[699   1]
 [  6 134]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      1.00       700
           1       0.99      0.96      0.97       140

    accuracy                           0.99       840
   macro avg       0.99      0.98      0.98       840
weighted avg       0.99      0.99      0.99       840


=== SVM (Normalized) ===
Accuracy: 0.9892857142857143

Confusion Matrix:
 [[700   0]
 [  9 131]]

Classification Report:
               precision    recall  f1-score   support

           0       0.99      1.00      0.99       700
           1       1.00      0.94      0.97       140

    accuracy                           0.99       840
   macro avg       0.99      0.97      0.98       840
weighted avg       0.99      0.99      0.99       840


=== KNN ===
Accuracy: 0.9845238095238096

Confusion Matrix:
 [[697   3]
 [ 10 130]]

Classification Re

In [2]:
import os
import cv2
import numpy as np

# Extra features
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops, hog

# ML tools
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# ===============================
# Parameter LBP
# ===============================
radius = 3
n_points = 8 * radius

# ===============================
# Fungsi ekstraksi fitur
# ===============================
def extract_features(image):
    # Resize agar HOG stabil
    image = cv2.resize(image, (128, 128))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # ======================
    # LBP
    # ======================
    lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
    lbp_hist, _ = np.histogram(
        lbp.ravel(),
        bins=np.arange(0, n_points + 3),
        range=(0, n_points + 2)
    )
    lbp_hist = lbp_hist / (lbp_hist.sum() + 1e-6)

    # ======================
    # GLCM
    # ======================
    glcm = graycomatrix(
        gray,
        distances=[1],
        angles=[0],        # 0 derajat
        levels=256,
        symmetric=True,
        normed=True
    )

    glcm_features = [
        graycoprops(glcm, 'contrast')[0,0],
        graycoprops(glcm, 'dissimilarity')[0,0],
        graycoprops(glcm, 'homogeneity')[0,0],
        graycoprops(glcm, 'energy')[0,0],
        graycoprops(glcm, 'correlation')[0,0]
    ]

    # ======================
    # HOG
    # ======================
    hog_feat = hog(
        gray,
        orientations=8,
        pixels_per_cell=(32, 32),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        feature_vector=True
    )

    # ======================
    # Gabungkan semua fitur
    # ======================
    features = np.hstack([lbp_hist, glcm_features, hog_feat])
    return features

# ===============================
# Load dataset
# ===============================
def load_dataset(folder_tb, folder_normal):
    X, y = [], []

    for img_name in os.listdir(folder_tb):
        img_path = os.path.join(folder_tb, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            X.append(extract_features(img))
            y.append(1)  # TB

    for img_name in os.listdir(folder_normal):
        img_path = os.path.join(folder_normal, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            X.append(extract_features(img))
            y.append(0)  # Normal

    return np.array(X), np.array(y)

# ===============================
# Tentukan folder dataset
# ===============================
folder_tb = r"D:\project\python\2025\tbcdetect\_data_shenzhen\train\tb"
folder_normal = r"D:\project\python\2025\tbcdetect\_data_shenzhen\train\normal"

print("[INFO] Loading dataset...")
X, y = load_dataset(folder_tb, folder_normal)
print("[INFO] Dataset loaded:", X.shape, "samples")

# ===============================
# Split train/test
# ===============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)




[INFO] Loading dataset...
[INFO] Dataset loaded: (662, 319) samples


In [ ]:
# ===============================
# StandardScaler
# ===============================
scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

# ===============================
# PCA → Kurangi fitur jadi 100
# ===============================
print("[INFO] Applying PCA...")
pca = PCA(n_components=100)
X_train_p = pca.fit_transform(X_train_s)
X_test_p = pca.transform(X_test_s)

# ===============================
# SVM
# ===============================
print("[INFO] Training SVM...")
svm = SVC(kernel='rbf', C=10, gamma='scale')
svm.fit(X_train_p, y_train)

# ===============================
# Evaluasi
# ===============================
y_pred = svm.predict(X_test_p)

print("\n========== HASIL AKHIR ==========")
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

In [5]:
import pickle
# ========================================================
# SIMPAN MODEL KE PICKLE
# ========================================================
model_pack = {
    "model": svm,
    "scaler": scaler,
    "radius": radius,
    "n_points": n_points
}

with open("tb_classifier_svm.pkl", "wb") as f:
    pickle.dump(model_pack, f)

print("Model saved as tb_classifier.pkl")

Model saved as tb_classifier.pkl
